# 10차시 실습: AI 에이전트 심층 해부 (Code-as-Action)

## 1. 개요
이 실습에서는 단순한 챗봇과 달리, 스스로 사고하고 도구를 사용하여 복잡한 문제를 해결하는 AI 에이전트(AI Agent)의 구조를 `smolagents` 라이브러리를 통해 깊이 있게 학습합니다.

특히 최신 트렌드인 **Code-as-Action(코드 기반 행동)** 패러다임이 기존의 JSON 기반 도구 호출보다 왜 강력한지 직접 코드를 통해 확인합니다.

### 🎯 학습 목표
1. **Cognitive Architecture:** 에이전트의 4대 요소(Brain, Tool, Memory, Planning) 이해
2. **Code-as-Action:** LLM이 파이썬 코드를 작성하여 논리적 문제(반복, 조건문)를 해결하는 과정 관찰
3. **Trace Analysis:** 에이전트의 사고 과정(Thought -> Code -> Observation) 로그 분석

## 2. 환경 설정 및 라이브러리 설치
가볍고 강력한 Hugging Face의 `smolagents`와 OpenRouter 연동을 위한 `litellm`을 설치합니다.

In [ ]:
!pip install smolagents litellm matplotlib pandas

In [ ]:
import os
from smolagents import CodeAgent, tool, LiteLLMModel

# OpenRouter API Key 설정 (실습 시 본인의 키를 입력하세요)
os.environ["OPENROUTER_API_KEY"] = "API Key를 입력해주세요!"

# 모델 설정: Qwen 2.5 7B 
model = LiteLLMModel(
    model_id="openrouter/qwen/qwen-2.5-7b-instruct",
    api_base="https://openrouter.ai/api/v1",
    api_key=os.environ["OPENROUTER_API_KEY"]
)

## 3. 도구(Tools) 정의: 에이전트의 손과 발
에이전트가 외부 세계와 상호작용하기 위한 함수들을 정의합니다. 
여기서는 단순 조회가 아닌, **데이터 처리 능력을 검증**하기 위해 주식 가격 조회 및 환율 계산 도구를 만듭니다.

In [3]:
@tool
def get_stock_price(ticker: str) -> float:
    """
    특정 주식 종목의 현재 가격을 조회합니다.
    
    Args:
        ticker: 주식 티커 기호 (예: 'AAPL', 'TSLA', 'GOOGL')
    """
    # 실습용 Mock Data
    prices = {
        "AAPL": 150.0,
        "TSLA": 200.0,
        "GOOGL": 2800.0
    }
    return prices.get(ticker.upper(), 0.0)

@tool
def get_exchange_rate(base: str, target: str) -> float:
    """
    두 통화 간의 환율 정보를 조회합니다.
    
    Args:
        base: 기준 통화 (예: 'USD')
        target: 대상 통화 (예: 'KRW')
    """
    if base.upper() == "USD" and target.upper() == "KRW":
        return 1350.0
    return 1.0

# 에이전트 생성 (CodeAgent: 파이썬 코드를 생성해 실행하는 에이전트)
agent = CodeAgent(tools=[get_stock_price, get_exchange_rate], model=model)

## 4. Code-as-Action의 강력함 체험
기존 JSON 방식의 에이전트는 "AAPL 가격 조회" -> "환율 조회" -> "계산"을 위해 여러 번 LLM을 호출해야 했습니다 (Multi-turn).
하지만 **Code Agent**는 **단 한 번의 파이썬 코드 생성**으로 이 복잡한 로직을 해결합니다.

In [4]:
query = "애플(AAPL) 주식 10주와 테슬라(TSLA) 주식 5주를 샀을 때, 총 평가액은 원화(KRW)로 얼마야?"

print(f"🙋 사용자 질문: {query}\n")
print("⏳ 에이전트가 생각 중입니다... (Code Generation)")

response = agent.run(query)

print(f"\n🤖 최종 답변: {response}")

🙋 사용자 질문: 애플(AAPL) 주식 10주와 테슬라(TSLA) 주식 5주를 샀을 때, 총 평가액은 원화(KRW)로 얼마야?

⏳ 에이전트가 생각 중입니다... (Code Generation)


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 애플(AAPL) 주식 10주와 테슬라(TSLA) 주식 5주를 샀을 때, 총 평가액은 원화(KRW)로 얼마야?                         │
│                                                                                                                 │
╰─ LiteLLMModel - openrouter/qwen/qwen-2.5-7b-instruct ───────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  aapl_price = get_stock_price('AAPL')                                                                             
  tsla_price = get_stock_price('TSLA')                                                                             
  print(aapl_price, tsla_price)                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
150.0 200.0

Out: None

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  aapl_quantity = 10                                                                                               
  tsla_quantity = 5                                                                                                
                                                                                                                   
  # Calculate total evaluation amount                                                                              
  total_evaluation_amount = (aapl_price * aapl_quantity) + (tsla_price * tsla_quantity)                            
  total_evaluation_amount_in_krw = get_exchange_rate('USD', 'KRW') * total_evaluation_amount                       
  final_answer(total_evaluation_amount_in_krw)                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 3375000.0

[Step 2: Duration 1.90 seconds| Input tokens: 4,415 | Output tokens: 210]


🤖 최종 답변: 3375000.0


### 🔍 실행 로그 분석 (Trace Analysis)
위 실행 결과에서 에이전트가 생성한 코드를 주목해 보세요. 아마도 아래와 유사할 것입니다.

```python
# 에이전트가 내부적으로 생성한 코드 예시
price_aapl = get_stock_price(ticker='AAPL')
price_tsla = get_stock_price(ticker='TSLA')
exchange_rate = get_exchange_rate(base='USD', target='KRW')

total_usd = (price_aapl * 10) + (price_tsla * 5)
total_krw = total_usd * exchange_rate
print(total_krw)
```

**시사점:**
1. **변수 활용:** `price_aapl`, `total_usd` 같은 변수를 사용하여 중간 값을 저장합니다.
2. **수학 연산:** LLM이 약한 사칙연산을 파이썬 인터프리터에게 위임하여 정확도를 보장합니다.
3. **효율성:** 여러 단계의 추론을 하나의 코드 블록으로 압축하여 실행 속도가 빠릅니다.

## 5. [심화] 반복문과 논리 처리
이번에는 에이전트가 **반복문(Loop)**을 사용할 수 있는지 테스트해 봅니다.

In [5]:
complex_query = "AAPL, TSLA, GOOGL 중에서 1주당 가격이 1000달러가 넘는 주식들의 가격 합계는 얼마야?"

response_complex = agent.run(complex_query)
print(f"\n🤖 심화 답변: {response_complex}")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ AAPL, TSLA, GOOGL 중에서 1주당 가격이 1000달러가 넘는 주식들의 가격 합계는 얼마야?                              │
│                                                                                                                 │
╰─ LiteLLMModel - openrouter/qwen/qwen-2.5-7b-instruct ───────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  apple_price = get_stock_price('AAPL')                                                                            
  tesla_price = get_stock_price('TSLA')                                                                            
  google_price = get_stock_price('GOOGL')                                                                          
                                                                                                                   
  stocks_above_1000 = []                                                                                           
                                                                                                                   
  if apple_price > 1000:                                                                                           
      stocks_above_1000.append(apple_price)                                                                        
  if tesla_price > 1000:                                                                                           
      stocks_above_1000.append(tesla_price)                                                                        
  if google_price > 1000:                                                                                          
      stocks_above_1000.append(google_price)                                                                       
                                                                                                                   
  total_price = sum(stocks_above_1000)                                                                             
  print(total_price)                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
2800.0

Out: None

[Step 1: Duration 2.57 seconds| Input tokens: 2,103 | Output tokens: 181]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(2800.0)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 2800.0

[Step 2: Duration 0.84 seconds| Input tokens: 4,600 | Output tokens: 228]


🤖 심화 답변: 2800.0


이번에는 CodeAgent가 단순 계산이나 반복문을 넘어서, **직접 파이썬 코드를 짜서 실험**해야만 풀 수 있는 문제를 내봅니다.

### 🧑‍💻 미션: "조건에 맞는 주식 포트폴리오 구성"
- **조건:**
    - AAPL, TSLA, GOOGL 세 종목 중에서
    - 1) 1주당 가격이 100달러 이상이고
    - 2) 총 예산 2000달러 내에서
    - 3) 각 종목을 최소 1주 이상 포함하여
    - 4) 살 수 있는 최대 주식 수와 각 종목별 매수 수량을 구하라

**힌트:** 단순 합산이 아니라, 반복문, 조건문, 그리디/브루트포스 등 다양한 파이썬 로직이 필요합니다. (CodeAgent가 직접 코드를 짜서 실험해야 정답을 찾을 수 있음)

---

In [7]:
# CodeAgent가 직접 다양한 파이썬 코드를 짜서 실험해야 하는 미션 쿼리
portfolio_query = "AAPL, TSLA, GOOGL 세 종목 중 1주당 가격이 100달러 이상이고, 총 예산 2000달러 내에서 각 종목을 최소 1주 이상 포함해 살 수 있는 최대 주식 수와 각 종목별 매수 수량을 알려줘. 해설도 부탁해"

print(f"🙋 사용자 미션: {portfolio_query}\n")
print("⏳ 에이전트가 복잡한 코드를 생성/실험 중입니다...")

portfolio_response = agent.run(portfolio_query)

print(f"\n🤖 미션 결과: {portfolio_response}")

🙋 사용자 미션: AAPL, TSLA, GOOGL 세 종목 중 1주당 가격이 100달러 이상이고, 총 예산 2000달러 내에서 각 종목을 최소 1주 이상 포함해 살 수 있는 최대 주식 수와 각 종목별 매수 수량을 알려줘. 해설도 부탁해

⏳ 에이전트가 복잡한 코드를 생성/실험 중입니다...


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ AAPL, TSLA, GOOGL 세 종목 중 1주당 가격이 100달러 이상이고, 총 예산 2000달러 내에서 각 종목을 최소 1주 이상     │
│ 포함해 살 수 있는 최대 주식 수와 각 종목별 매수 수량을 알려줘. 해설도 부탁해                                    │
│                                                                                                                 │
╰─ LiteLLMModel - openrouter/qwen/qwen-2.5-7b-instruct ───────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  AAPL_price = get_stock_price(ticker='AAPL')                                                                      
  TSLA_price = get_stock_price(ticker='TSLA')                                                                      
  GOOGL_price = get_stock_price(ticker='GOOGL')                                                                    
                                                                                                                   
  stock_prices = [AAPL_price, TSLA_price, GOOGL_price]                                                             
  tickers = ['AAPL', 'TSLA', 'GOOGL']                                                                              
  prices_more_than_100 = []                                                                                        
                                                                                                                   
  for price, stock in zip(stock_prices, tickers):                                                                  
      if price > 100:                                                                                              
          prices_more_than_100.append(price)                                                                       
                                                                                                                   
  if len(prices_more_than_100) == 0:                                                                               
      print("None of the stocks have a price above $100")                                                          
  else:                                                                                                            
      total_budget = 2000                                                                                          
      total_shares = int(total_budget / sum(prices_more_than_100))                                                 
                                                                                                                   
      # 最大化总份额                                                                                               
      max_shares_each_ticker = {}                                                                                  
      remaining_budget = total_budget                                                                              
      for price in prices_more_than_100:                                                                           
          shares_this_ticker = remaining_budget // price                                                           
          max_shares_each_ticker[tickers[prices_more_than_100.index(price)]] = shares_this_ticker                  
          remaining_budget -= shares_this_ticker * price                                                           
                                                                                                                   
      print("In the budget of $2000, the maximum number of shares that can be purchased is", total_shares)         
      print("Each stock should buy", max_shares_each_ticker)                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
In the budget of $2000, the maximum number of shares that can be purchased is 0
Each stock should buy {'AAPL': 13.0, 'TSLA': 0.0, 'GOOGL': 0.0}

Out: 50.0

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer({"AAPL": 13, "TSLA": 0, "GOOGL": 0})                                                                
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: {'AAPL': 13, 'TSLA': 0, 'GOOGL': 0}

[Step 4: Duration 2.20 seconds| Input tokens: 13,122 | Output tokens: 1,163]


🤖 미션 결과: {'AAPL': 13, 'TSLA': 0, 'GOOGL': 0}


In [ ]:
# 정답(예시, 실제 agent 답변과 다를 수 있음)
print("\n[정답 및 해설 예시]")
prices = {"AAPL": 150.0, "TSLA": 200.0, "GOOGL": 2800.0}
budget = 2000
# 1주당 100달러 이상 종목만
valid = {k: v for k, v in prices.items() if v >= 100}
# 각 종목 최소 1주씩 구매
min_cost = sum(valid.values())
if min_cost > budget:
    print("예산 부족으로 조건 충족 불가")
else:
    remain = budget - min_cost
    # 남은 예산으로 가장 저렴한 종목부터 최대 구매
    buy = {k: 1 for k in valid}
    items = sorted(valid.items(), key=lambda x: x[1])
    while True:
        bought = False
        for k, price in items:
            if remain >= price:
                buy[k] += 1
                remain -= price
                bought = True
        if not bought:
            break
    print(f"총 매수 주식 수: {sum(buy.values())}")
    print(f"종목별 매수 수량: {buy}")
    print(f"남은 예산: {remain}달러")
    print("(풀이: 각 종목 1주씩 산 뒤, 남은 예산으로 반복적으로 가장 싼 종목부터 추가 매수)")


[정답 및 해설 예시]
예산 부족으로 조건 충족 불가
